In [1]:
from nltk.corpus import WordNetCorpusReader as WNCR, wordnet as wn
import nltk
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

#nltk.download('universal_tagset')

WN16 = WNCR('/Users/ddobolyi/Downloads/textagon-portable/external/extracted/wordnet-1.6/dict', 
            '/Users/ddobolyi/Downloads/textagon-portable/external/extracted/wordnet-1.6/dict')

synList = ET.parse('/Users/ddobolyi/Downloads/textagon-portable/external/extracted/wn-domains/wn-affect-1.1/a-synsets.xml').getroot()
posTags = [child.tag for child in synList]
allPOS = [pos.split('-')[0] for pos in posTags]
WNA11Synsets = {}

for eachPOS in allPOS:
    WNA11Synsets[eachPOS] = pd.DataFrame([item.attrib for item in synList.findall(eachPOS + '-syn-list/' + eachPOS + '-syn')])
    WNA11Synsets[eachPOS]['intid'] = WNA11Synsets[eachPOS]['id'].str[2:].astype('int')

def GetWNAffect(word):

    recognizedPOS = {'adj': wn.ADJ, 'adv': wn.ADV, 'noun': wn.NOUN, 'verb': wn.VERB}
    mappedPOS = nltk.map_tag('en-ptb', 'universal', nltk.pos_tag([word])[0][1]).lower()

    if mappedPOS in recognizedPOS:
        nltkPOS = recognizedPOS[mappedPOS]

    synsets = WN16.synsets(word, nltkPOS)
    offsets = [i.offset() for i in synsets]

    #print(synsets)
    #print(offsets)

    if mappedPOS != 'noun':
        checkMatch = WNA11Synsets[mappedPOS][WNA11Synsets[mappedPOS].intid.isin(offsets)]['noun-id'].values[0]
    else:
        checkMatch = WNA11Synsets[mappedPOS][WNA11Synsets[mappedPOS].intid.isin(offsets)]['id'].values[0]

    return(WNA11Synsets['noun'][WNA11Synsets['noun'].id.values == checkMatch].categ.values[0])

In [2]:
GetWNAffect('angry') # need to test against WNAffect more broadly

'anger'